In [2]:
import io
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, SimpleRNN, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
# fix random seed for reproducibility
np.random.seed(42)

###########################################################################
#generate training/test data
########################################################################### 
br_file = io.open("positive.txt",'r',encoding="latin-1")
bi_file = io.open("negative.txt",'r',encoding="latin-1")



br_tweets = []
bi_tweets = []

for _ in range(250000):
    br_tweets.append([br_file.readline().strip("\n"),0])
    bi_tweets.append([bi_file.readline().strip("\n"),1])
    
tweets = br_tweets+bi_tweets
np.random.shuffle(tweets)

#1350000
#945000
tweets_train = np.array(tweets[:300000])
tweets_test = np.array(tweets[300000:])

X_train = tweets_train[:,0]
Y_train = to_categorical(np.array([int(i) for i in tweets_train[:,1]],dtype=np.int))
X_test = tweets_test[:,0]
Y_test = to_categorical(np.array([int(i) for i in tweets_test[:,1]],dtype=np.int))

test_tweets = ["Fuck you mom","I love you mom","I hate homework","I enjoy homework", "Life sucks","Life is good",
               "Jesus loves me","Put a ring on it","Mercedes Benz: $20,000",
                "The decision on Sergeant Bergdahl is a complete and total disgrace to our Country and to our Military.",
                "Does anybody really believe that a reporter, who nobody ever heard of, went to his mailbox and found my tax returns? @NBCNews FAKE NEWS!",
                "After Turkey call I will be heading over to Trump National Golf Club, Jupiter, to play golf (quickly) with Tiger Woods and Dustin Johnson. Then back to Mar-a-Lago for talks on bringing even more jobs and companies back to the USA!",
                "“The President has accomplished some absolutely historic things during this past year.” Thank you Charlie Kirk of Turning Points USA. Sadly, the Fake Mainstream Media will NEVER talk about our accomplishments in their end of year reviews. We are compiling a long & beautiful list.",
                "Crazy Joe Scarborough and dumb as a rock Mika are not bad people, but their low rated show is dominated by their NBC bosses. Too bad!",
                "Robert Pattinson should not take back Kristen Stewart. She cheated on him like a dog & will do it again--just watch. He can do much better!"]

#########################################################################
#bag of words
#########################################################################

vectorizer = CountVectorizer( )
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)
test_tweets_counts = vectorizer.transform(test_tweets)


print("SUCCESS")


SUCCESS


"\ntop_words = 10000\nmax_tweet_length = 50\n\ntokenizer = Tokenizer(num_words = top_words)\ntokenizer.fit_on_texts(X_train)\ntokenizer.fit_on_texts(X_test)\ntokenizer.fit_on_texts(test_tweets)\nsequences = tokenizer.texts_to_sequences(X_train)\nword_index = tokenizer.word_index\nprint ('Found %s unique tokens.' % len(word_index))\n\nX_train = sequence.pad_sequences(sequences, maxlen=max_tweet_length)\nY_train = np.asarray(Y_train)\n\n\n\n\n\n#X_test = sequence.pad_sequences(X_test_counts, maxlen=max_tweet_length)\n"

In [ ]:
import keras.optimizers


model = Sequential() 
model.add( Dense(250, activation="selu", input_dim= X_train_counts.shape[1]) )
model.add( Dense(units=2, activation="softmax"))

sgd = keras.optimizers.SGD(lr=0.0001*400)
model.compile(optimizer=sgd,loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

history = model.fit(X_train_counts, Y_train, epochs=9, batch_size=32, validation_split = 0.2)
model.save_weights('pos_neg_NN_model.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 250)               52256750  
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 502       
Total params: 52,257,252
Trainable params: 52,257,252
Non-trainable params: 0
_________________________________________________________________
None
Train on 240000 samples, validate on 60000 samples
Epoch 1/9
 29120/240000 [==>...........................] - ETA: 1:03:48 - loss: 0.5790 - acc: 0.7077

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) +1)

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.load_weights('pos_neg_NN_model.h5')
print("Classification Error on Test Set: ",model.evaluate(X_test_counts,Y_test,verbose=0))

In [ ]:



predictions = model.predict(test_tweets_counts)
for i in range(len(predictions)):
    print(test_tweets[i],"\n",predictions[i])